<a href="https://colab.research.google.com/github/canamac/zeka-can/blob/main/lojistik_regrasyon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np
import math
import requests

# CoinGecko API'den veri çekme
url = "https://api.coingecko.com/api/v3/coins/bitcoin/market_chart"
params = {
    "vs_currency": "usd",
    "days": "1",  # Son 1 gün
    "interval": "daily"  # Günlük veri kullanımı
}

response = requests.get(url, params=params)
data_json = response.json()

# Veriyi kontrol et ve CSV'ye kaydet
if "prices" in data_json:
    prices = data_json["prices"]
    df = pd.DataFrame(prices, columns=["timestamp", "close"])
    df["timestamp"] = pd.to_datetime(df["timestamp"], unit="ms")  # Zaman damgasını dönüştür
    df.to_csv("price_data.csv", index=False)
    print("Veri başarıyla indirildi ve 'price_data.csv' olarak kaydedildi.")
else:
    raise ValueError("API yanıtında 'prices' verisi bulunamadı.")

# CSV'den veri yükleme
data = pd.read_csv("price_data.csv")

# Eksik veya hatalı verileri temizleme
data.dropna(inplace=True)

# Hareketli ortalamalar ve göstergeleri hesaplama
ema_length = 20
rsi_length = 14
bollinger_length = 20
bollinger_std_dev = 2.0
rsi_ema_length = 200
bollinger_ema_length = 200

# EMA hesaplama
data['EMA'] = data['close'].ewm(span=ema_length, adjust=False).mean()

# RSI hesaplama
delta = data['close'].diff(1)
gain = (delta.where(delta > 0, 0)).rolling(window=rsi_length).mean()
loss = (-delta.where(delta < 0, 0)).rolling(window=rsi_length).mean()
rs = gain / loss
data['RSI'] = 100 - (100 / (1 + rs))

# Bollinger Bantları hesaplama
data['MiddleBand'] = data['close'].rolling(window=bollinger_length).mean()
data['StdDev'] = data['close'].rolling(window=bollinger_length).std()
data['UpperBand'] = data['MiddleBand'] + (bollinger_std_dev * data['StdDev'])
data['LowerBand'] = data['MiddleBand'] - (bollinger_std_dev * data['StdDev'])

# EMA 200 RSI hesaplama
data['RSI_EMA'] = data['RSI'].ewm(span=rsi_ema_length, adjust=False).mean()

# EMA 200 Bollinger hesaplama
data['Bollinger_EMA'] = data['MiddleBand'].ewm(span=bollinger_ema_length, adjust=False).mean()

# Alım/Satım Sinyalleri
buy_signal = (data['close'] > data['EMA']) & (data['RSI'] < 30) & (data['close'] < data['LowerBand'])
sell_signal = (data['close'] < data['EMA']) & (data['RSI'] > 70) & (data['close'] > data['UpperBand'])

data['BuySignal'] = buy_signal
data['SellSignal'] = sell_signal

# Nash Dengesi
nash_equilibrium = ~(buy_signal & sell_signal)
data['NashEquilibrium'] = nash_equilibrium

# Lojistik Regresyon Hesaplamaları
ema_prob = (data['close'] - data['EMA']) / data['close']
rsi_prob = (data['RSI'] - 50) / 50
bollinger_prob = (data['close'] - data['MiddleBand']) / (data['UpperBand'] - data['LowerBand'])

data['CombinedProbability'] = (ema_prob + rsi_prob + bollinger_prob) / 3
data['LogisticProbability'] = 1 / (1 + np.exp(-data['CombinedProbability']))

# Nihai sinyal hesaplama
data['FinalBuySignal'] = data['BuySignal'] & (data['LogisticProbability'] > 0.5) & data['NashEquilibrium']
data['FinalSellSignal'] = data['SellSignal'] & (data['LogisticProbability'] < 0.5) & data['NashEquilibrium']

# Çıktıyı yazdırma
data.to_csv("processed_signals.csv", index=False)
print(data[['close', 'FinalBuySignal', 'FinalSellSignal']].tail())


Veri başarıyla indirildi ve 'price_data.csv' olarak kaydedildi.
          close  FinalBuySignal  FinalSellSignal
0  96558.237625           False            False
1  96922.495300           False            False
